In [ ]:
from google.colab import drive
import numpy as np
drive.mount('/content/drive')
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

# Load the Policy Metadata file
csv_file = 'directory of policy meta data file'

# Create a dictionary to map ID to state and action year
id_to_state_and_year = dict(zip(metadata['ID'].astype(str), zip(metadata['State / Jurisdiction'], metadata['Action_Year'])))

# Function to match txt file names with state and action year
def get_state_and_year_for_file(file_name, id_to_state_and_year):
    # Handle cases with letters in file names (e.g., 11a, 11b)
    base_file_name = ''.join([char for char in file_name if char.isdigit()])
    return id_to_state_and_year.get(base_file_name)

text_files_directory = "Directory containing the text files"
# Create a DataFrame to store file names, their states, and action years
file_state_year_data = []
for file_name in os.listdir(text_files_directory):
    if file_name.endswith('.txt'):
        state_year_info = get_state_and_year_for_file(file_name.split('.')[0], id_to_state_and_year)
        if state_year_info:
            state, action_year = state_year_info
            file_state_year_data.append({'File Name': file_name, 'State': state, 'Action Year': action_year})

# Convert to DataFrame
file_state_df = pd.DataFrame(file_state_year_data)
file_state_df['Action Year']=file_state_df['Action Year'].astype("int")

print(file_state_df)

    File Name          State  Action Year
0       3.txt         Alaska         2016
1       4.txt         Alaska         2018
2       7.txt       Arkansas         2017
3       8.txt       Arkansas         2015
4      10.txt        Arizona         2021
..        ...            ...          ...
151   147.txt      Wisconsin            0
152   149.txt  West Virginia            0
153    62.txt       Michigan         1987
154    40.txt        Indiana            0
155    87.txt         Nevada            0

[156 rows x 3 columns]


In [ ]:
# Preprocessing
import nltk
import re
from collections import Counter
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
class TextProcessor:
    def __init__(self, input_corpus: list):
        self.processed_corpus = input_corpus

    def remove_digits_and_punctuation(self):
        regex_pattern = re.compile(r'[0-9]+|[^\w\s]')
        self.processed_corpus = [regex_pattern.sub(" ", row) for row in self.processed_corpus]

    def to_lowercase(self):
        self.processed_corpus = [row.lower() for row in self.processed_corpus]

    def remove_stop_words(self):
        stops = set(stopwords.words('english')) - {
            "shan't", "couldn't", "against", "shouldn't", "can't",
            "needn't", "should've", "not", "mustn't", "will"
        }
        stops.update(string.ascii_lowercase)
        stops.update(['ii', 'iii', 'iv'])
        self.processed_corpus = [
            " ".join([token for token in row.split() if token not in stops])
            for row in self.processed_corpus
        ]

    def remove_common_words(self):
        counter = Counter(" ".join(self.processed_corpus).split())
        most_common = set(word for word, count in counter.most_common(10))
        self.processed_corpus = [
            " ".join(token for token in row.split() if token not in most_common)
            for row in self.processed_corpus
        ]

    def process(self):
        self.remove_digits_and_punctuation()
        self.to_lowercase()
        self.remove_stop_words()
        self.remove_common_words()
        return self.processed_corpus

def read_text_files(directory):
    file_contents = {}
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    file_contents[filename] = file.read()
            except UnicodeDecodeError:
                try:
                    with open(file_path, 'r', encoding='ISO-8859-1') as file:
                        file_contents[filename] = file.read()
                except Exception as e:
                    print(f"Failed to read {filename}: {e}")
    return file_contents
# Load and preprocess the text files
file_directory = "Directory containing the text files"
file_contents = read_text_files(file_directory)
text_processor = TextProcessor(list(file_contents.values()))
preprocessed_texts = text_processor.process()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
file_to_state = dict(zip(file_state_df['File Name'], file_state_df['State']))
file_to_year_map = dict(zip(file_state_df['File Name'], file_state_df['Action Year']))
file_to_state_map = dict(zip(file_state_df['File Name'], file_state_df['State']))

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install gensim

In [ ]:
!pip install fasttext

In [ ]:
import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')

'cc.en.300.bin'

In [ ]:
import zipfile
from gensim.scripts.glove2word2vec import glove2word2vec

glove_zip_path = '/content/drive/MyDrive/Policy_comparison/glove.6B.zip'
glove_folder = '/content/glove.6B'
with zipfile.ZipFile(glove_zip_path, 'r') as zip_ref:
    zip_ref.extractall(glove_folder)
glove_input_file = os.path.join(glove_folder, 'glove.6B.300d.txt')
word2vec_output_file = os.path.join(glove_folder, 'glove.6B.300d.word2vec.txt')
glove2word2vec(glove_input_file, word2vec_output_file)

<ipython-input-9-80b4c3d88f92>:10: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400001, 300)

In [ ]:
from collections import defaultdict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
import torch
import fasttext
from gensim.models import KeyedVectors
from collections import defaultdict
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
from gensim.models import LdaMulticore
from gensim.models import KeyedVectors
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
import fasttext

# Define functions for each model
def get_tfidf_nmf_embeddings(texts, n_components=10, ngram_range=(1, 1)):
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_df=0.95, min_df=2, lowercase=False)
    tfidf = vectorizer.fit_transform(texts)
    nmf = NMF(n_components=n_components, random_state=42)
    nmf_features = nmf.fit_transform(tfidf)
    return nmf_features

def get_sentence_transformer_embeddings(texts):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(texts, convert_to_tensor=False)
    pca = PCA(n_components=10)
    reduced_embeddings = pca.fit_transform(embeddings)
    return reduced_embeddings

def get_bert_embeddings(texts):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    outputs = model(**encoded_input)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

def get_fasttext_embeddings(texts):
    model = fasttext.load_model('cc.en.300.bin')
    return np.array([model.get_sentence_vector(text) for text in texts])

def get_glove_embeddings(texts):
    glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
    embeddings = [np.mean([glove_model[word] for word in text.split() if word in glove_model], axis=0) for text in texts if text.split()]
    pca = PCA(n_components=10)
    reduced_embeddings = pca.fit_transform(embeddings)
    return reduced_embeddings

# Function to find similar policies
def find_similar_policies_A(single_policy, all_policies, threshold = 0.5):
    models = {
        'TFIDF-NMF-1': lambda texts: get_tfidf_nmf_embeddings(texts, 10, (1, 1)),
        'TFIDF-NMF-2': lambda texts: get_tfidf_nmf_embeddings(texts, 10, (1, 2)),
        'SentenceTransformer': get_sentence_transformer_embeddings,
        'BERT': get_bert_embeddings,
        'FastText': get_fasttext_embeddings,
        'GloVe': get_glove_embeddings
    }

    all_policies_filtered = [p for p in all_policies if p != single_policy]
    results = defaultdict(list)

    for name, model_func in models.items():
        embeddings = model_func([single_policy] + all_policies_filtered)
        single_embedding = embeddings[0]
        other_embeddings = embeddings[1:]
        similarities = cosine_similarity([single_embedding], other_embeddings)[0]

        for idx, similarity_score in enumerate(similarities):
            if similarity_score > threshold:
                results[(all_policies_filtered[idx], similarity_score)].append(name)

    option_a = {processed_to_original[content]: list(models.keys()) for content, models in model_votes.items() if len(models) >= 2}
    option_c = {processed_to_original[content]: model_names for content, model_names in top_pairs_per_model.items() if len(set(model_names)) >= 2}

    return option_a, option_c

original_texts = list(file_contents.values())  # List of original texts
option_a, option_c = find_similar_policies(single_policy, preprocessed_texts, original_texts, threshold=0.8, top_n=10)

